# Пример агента за резервацију хотела и летова

Ово решење ће вам помоћи да резервишете авионске карте и хотел. Сценарио је путовање од Лондон Хитроув LHR 20. фебруара 2024. до Њујорка JFK са повратком 27. фебруара 2025. летећи економском класом само са British Airways. Желим боравак у хотелу Hilton у Њујорку, молим обезбедите трошкове за лет и хотел.


# Иницијализујте Azure AI Agent Service и преузмите конфигурационе информације из **.env**

### **.env** 

Креирајте .env фајл

**.env** садржи конекциони низ Azure AI Agent Service, модел који користи AOAI, и одговарајући Google API Search сервиса API, ENDPOINT, итд.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Име вашег Azure AI Agent Service Model Deployment-а"

[**ВАЖНО**] Биће вам потребан модел са 100.000 Rate Limit (токена у минути) и Rate Limit од 600 (захтева у минути)

  Можете добити модел у Microsoft Foundry - Model and Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Конекциони низ вашег Azure AI Agent Service пројекта"

  Конекциони низ пројекта можете добити у прегледу вашег пројекта на AI ​​Foundry Portal екрану.

- **SERPAPI_SEARCH_API_KEY** = "ВАШ SERPAPI Search API КЉУЧ"
- **SERPAPI_SEARCH_ENDPOINT** = "ВАША SERPAPI Search Endpoint адреса"

Да бисте добили Име Model Deployment и Project Connection String за Azure AI Agent Service, потребно је да креирате Azure AI Agent Service. Препоручује се да користите [ову шаблону](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) за њену директну креирање （***Напомена:*** Azure AI Agent Service је тренутно доступан само у ограниченом региону. Препоручује се да погледате [овaj линк](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) за подешавање региона)

Агент треба приступ SERPAPI-ју. Препоручује се да се региструјете преко [овог линка](https://serpapi.com/searches). Након регистрације, можете добити јединствени API КЉУЧ и ENDPOINT.


# Подешавање 

Да бисте покренули овај бележник, мораћете да се уверите да сте инсталирали потребне библиотеке покретањем `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Ваша верзија Semantic Kernel-а треба да буде најмање 1.27.2.


Учитајте ваш .env фајл са подешавањима и ресурсима, молимо вас да осигурате да сте додали ваше кључеве и подешавања и да сте креирали локални .env фајл.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Пријавите се у Azure

Сада морате да се пријавите у Azure. Отворите терминал и покрените следећу команду:

```bash
az login
```

Ова команда ће вас упитати да унесете своје Azure акредитиве, омогућавајући сервису Azure AI Agent да исправно функционише.


# Објашњење:
Ово је променљива која чува API кључ за приступ SERP (страница са резултатима претраге) API сервису. API кључ је јединствени идентификатор који се користи за аутентификацију захтева везаних за ваш налог.

Сврха: Сврха овог реда је да чува API кључ у променљивој како би се могао користити за аутентификацију захтева према SERP API сервису. API кључ је неопходан за приступ сервису и обављање претрага.
Како добити SERP API кључ: Да бисте добили SERP API кључ, пратите ове опште кораке на https://serpapi.com (тачни кораци могу варирати у зависности од специфичног SERP API сервиса који користите):

Изаберите SERP API сервис: Постоји неколико SERP API сервиса, као што су SerpAPI, Google Custom Search JSON API, и други. Изаберите онај који најбоље одговара вашим потребама.

Региструјте налог: Идите на веб страницу изабраног SERP API сервиса и региструјте налог. Можда ћете морати да доставите неке основне информације и потврдите своју е-маил адресу.

Креирајте API кључ: Након регистрације, пријавите се на свој налог и идите у API одељак или контролни панел. Потражите опцију за креирање или генерисање новог API кључа.
Копирајте API кључ у свој .env фајл.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Објашњење:
BASE_URL: Ово је променљива која чува базни URL за SERP API крајњу тачку. Назив променљиве BASE_URL је конвенција која се користи да означи да је овај URL почетна тачка за прављење API захтева.
'https://serpapi.com/search':

Ово је стварни URL низ додељен променљивој BASE_URL. Представља крајњу тачку за извршавање претраживачких упита користећи SERP API.

# Сврха:
Сврха овог реда је да дефинише константу која чува базни URL за SERP API. Овај URL ће се користити као почетна тачка за конструкцију API захтева за извођење претраживачких операција.

# Коришћење:
Дефинисањем базног URL у променљивој, лако га можете поново користити у целом коду кад год треба да направите захтеве ка SERP API-ју. Ово чини ваш код лакшим за одржавање и смањује ризик од грешака услед ручног уноса URL-а на више места. Тренутни пример је https://serpapi.com/search?engine=bing који користи Bing претраживачки API. Различити API могу се изабрати на https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Објашњење:

Овде се налази код вашег плагина.

Дефиниција класе: `class BookingPlugin`: Дефинише класу BookingPlugin која садржи методе за резервацију хотела и летова.

Метод за резервацију хотела:

- `@kernel_function(description="booking hotel")`: Декоратор који описује функцију као кернел функцију за резервацију хотела.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Дефинише метод за резервацију хотела са анотиранм параметрима и типом повратне вредности.

Метод конструише речник параметара за захтев резервације хотела и шаље GET захтев ка SERP API-ју. Проверава статус одговора и враћа информације о хотелу ако је успешан, или None ако је захтев неуспешан.

Метод за резервацију лета: 

- `@kernel_function(description="booking flight")`: Декоратор који описује функцију као кернел функцију за резервацију летова.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Дефинише метод за резервацију летова са анотиранм параметрима и типом повратне вредности.

Метод конструише речнике параметара за захтеве излазног и повратног лета и шаље GET захтеве ка SERP API-ју. Проверава статус одговора и додаје информације о лету у резултатски низ ако је успешан, или исписује поруку о грешци ако је захтев неуспешан. Метод враћа резултатски низ који садржи информације о лету.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Објашњење:
Import statements: Импортују се потребни модули за Azure акредитиве, AI агента, садржај поруке за ћаскање, улогу аутора и декоратор функције кернела.

Асинхрони менаџер контекста: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Ово поставља асинхрони менаџер контекста за руковање Azure акредитивима и креирање AI агента клијента.

Име агента и упутства:
- `AGENT_NAME = "BookingAgent"`: Дефинише име агента.
- `AGENT_INSTRUCTIONS = """..."""`: Пружа детаљна упутства агенту о томе како да рукује захтевима за резервације.

Креирање дефиниције агента: `agent_definition = await client.agents.create_agent(...)`: Креира дефиницију агента са назначеним моделом, именом и упутствима.

Креирање AzureAI агента: `agent = AzureAIAgent(...)`: Креира AzureAI агента користећи клијента, дефиницију агента и дефинисани додатак.

Креирање нити: `thread: AzureAIAgentThread | None = None`: Креира нит за агента. Није неопходно прво креирати нит — ако је вредност `None` прослеђена, нова нит ће бити креирана током првог позива и враћена као део одговора.

Кориснички уноси: `user_inputs = ["..."]`: Дефинише листу корисничких уноса које агент обрађује.

У блоку finally, избришите нит и агента ради чишћења ресурса.


# Аутентификација

Класа `DefaultAzureCredential` је део Azure SDK за Питон. Она пружа подразумевани начин за аутентификацију са Azure сервисима. Покушава да се аутентификује коришћењем више метода у одређеном редоследу, као што су променљиве окружења, управљани идентитет и Azure CLI акредитиви.

Асинхроне операције: aio модул указује да класа DefaultAzureCredential подржава асинхроне операције. То значи да је можете користити са asyncio за извођење не-блокирајућих захтева за аутентификацију.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Одрицање од одговорности**:
Овај документ је преведен помоћу АИ сервиса за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако тежимо прецизности, молимо имајте у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом матичном језику треба сматрати ауторитетом. За критичне информације препоручује се професионално људско превођење. Не преузимамо одговорност за било каква неспоразумевања или погрешна тумачења настала употребом овог превода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
